## 作業
礙於不是所有同學都有 GPU ，這邊的範例使用的是簡化版本的 ResNet，確保所有同學都能夠順利訓練!


最後一天的作業請閱讀這篇非常詳盡的[文章](https://blog.gtwang.org/programming/keras-resnet-50-pre-trained-model-build-dogs-cats-image-classification-system/)，基本上已經涵蓋了所有訓練　CNN 常用的技巧，請使用所有學過的訓練技巧，盡可能地提高 Cifar-10 的 test data 準確率，截圖你最佳的結果並上傳來完成最後一次的作業吧!

另外這些技巧在 Kaggle 上也會被許多人使用，更有人會開發一些新的技巧，例如使把預訓練在 ImageNet 上的模型當成 feature extractor 後，再拿擷取出的特徵重新訓練新的模型，這些技巧再進階的課程我們會在提到，有興趣的同學也可以[參考](https://www.kaggle.com/insaff/img-feature-extraction-with-pretrained-resnet)

In [1]:
import os
import numpy as np
from keras.datasets import cifar10
from keras.utils import np_utils
from keras.layers import Input, Dense, BatchNormalization, Dropout, Conv2D, MaxPooling2D, Flatten
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
np.random.seed(10)

Using TensorFlow backend.


In [2]:
### load data
(x_train_ori, y_train_ori), (x_test_ori, y_test_ori) = cifar10.load_data()
print('x_train:', x_train_ori.shape)
print('y_train:', y_train_ori.shape)
print('x_test:', x_test_ori.shape)
print('y_test:', y_test_ori.shape)

x_train: (50000, 32, 32, 3)
y_train: (50000, 1)
x_test: (10000, 32, 32, 3)
y_test: (10000, 1)


In [3]:
### preprocessing
# normalize
x_train = x_train_ori / 255
x_test = x_test_ori / 255

# label: ont-hot encoding
y_train = np_utils.to_categorical(y_train_ori)
y_test = np_utils.to_categorical(y_test_ori)

In [4]:
### define network
def build_network(input_shape, output_units, num_neurons=[512], num_filters=[32, 64]):
    input_layer = Input(input_shape)
    x = input_layer
    
    for filters in num_filters:
        x = Conv2D(filters=filters, kernel_size=(3, 3), padding='same', activation='relu')(x)
        x = BatchNormalization()(x)
        x = Conv2D(filters=filters, kernel_size=(3, 3), activation='relu')(x)
        x = BatchNormalization()(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)
        x = Dropout(0.25)(x)
    x = Flatten()(x)
    
    for units in num_neurons:
        x = Dense(units=units, activation='relu')(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        
    out = Dense(units=output_units, activation='softmax')(x)
    
    model = Model(inputs=[input_layer], outputs=[out])
    print(model.summary())
    return model

model = build_network(x_train.shape[1:], 10)

W0809 00:20:10.283389 17160 deprecation_wrapper.py:119] From c:\users\user\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0809 00:20:10.934851 17160 deprecation_wrapper.py:119] From c:\users\user\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0809 00:20:10.944858 17160 deprecation_wrapper.py:119] From c:\users\user\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0809 00:20:11.051934 17160 deprecation_wrapper.py:119] From c:\users\user\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0809 00:20:11.053934 17160 deprecation_wrapper.py:119] From

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
batch_normalization_2 (Batch (None, 30, 30, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 15, 15, 32)        0         
__________

In [5]:
LEARNING_RATE = 1e-3
EPOCHS = 20
BATCH_SIZE = 512

optimizer = Adam(lr=LEARNING_RATE)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

earlystop = EarlyStopping(monitor="val_loss", patience=5)
reduce_lr = ReduceLROnPlateau(factor=0.8, min_lr=1e-6, monitor='val_loss', patience=3)
callbacks = [earlystop, reduce_lr]

history = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=EPOCHS, batch_size=BATCH_SIZE,
                    callbacks=callbacks)

W0809 00:20:45.423339 17160 deprecation_wrapper.py:119] From c:\users\user\anaconda3\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0809 00:20:45.773588 17160 deprecation.py:323] From c:\users\user\anaconda3\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 50000 samples, validate on 10000 samples
Epoch 1/20
50000/50000 [==============================] - 112s 2ms/step - loss: 1.8352 - acc: 0.4218 - val_loss: 1.4855 - val_acc: 0.5033
Epoch 2/20
50000/50000 [==============================] - 81s 2ms/step - loss: 1.2331 - acc: 0.5691 - val_loss: 1.2375 - val_acc: 0.5709
Epoch 3/20
50000/50000 [==============================] - 81s 2ms/step - loss: 1.0236 - acc: 0.6412 - val_loss: 1.0376 - val_acc: 0.6363
Epoch 4/20
50000/50000 [==============================] - 82s 2ms/step - loss: 0.9069 - acc: 0.6821 - val_loss: 0.9130 - val_acc: 0.6815
Epoch 5/20
50000/50000 [==============================] - 81s 2ms/step - loss: 0.8216 - acc: 0.7111 - val_loss: 0.7820 - val_acc: 0.7319
Epoch 6/20
50000/50000 [==============================] - 81s 2ms/step - loss: 0.7510 - acc: 0.7346 - val_loss: 0.7499 - val_acc: 0.7390
Epoch 7/20
50000/50000 [==============================] - 82s 2ms/step - loss: 0.6946 - acc: 0.7535 - val_loss: 0.7263 - val_ac